# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [735]:
%reload_ext dotenv
%dotenv

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [736]:
import os
from glob import glob

parquet_files = glob(os.path.join(PRICE_DATA, "**/*parquet/part.0.parquet"))

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [737]:
import dask.dataframe as dd

# Read the data into a Dask dataframe
# set the index value to 'ticker'

dd_px = dd.read_parquet(parquet_files).set_index('ticker')

c:\Users\njay1\miniconda3\envs\dsi_participant\lib\site-packages\pandas\core\frame.py:717: DeprecationWarning: Passing a BlockManager to DataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
c:\Users\njay1\miniconda3\envs\dsi_participant\lib\site-packages\dask\dataframe\core.py:5517: UserWarning: New index has same name as existing, this is a no-op.
  warnings.warn(


In [740]:
import numpy as np

# Add the columns 'Close_lag_1', and 'Adj_Close_lag_1', based on the 'Close' and 'Adj Close' values from previous day for each row
# Use the .assign() function to define the columns, and .shift(1) to get the value from the previous date, which will be the value
# the previous row

dd_feat = (
    dd_px.groupby('ticker', group_keys=False
        ).apply(lambda x: x.assign(Close_lag_1 = x['Close'].shift(1) )
    ).assign(
        Adj_Close_lag_1 = lambda x : x['Adj Close'].shift(1)
    ).assign(
        returns = lambda x: (x['Adj Close']/x['Adj_Close_lag_1'] - 1)
    ).assign(
        high_low_range = lambda x: (x['High']-x['Low'])
    ))



C:\Users\njay1\AppData\Local\Temp\ipykernel_21100\3757648734.py:8: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_px.groupby('ticker', group_keys=False


In [742]:
dd_feat


,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag_1,Adj_Close_lag_1,returns,high_low_range
npartitions=11207,,,,,,,,,,,,,,
,datetime64[ns],float64,float64,float64,float64,float64,int64,object,object,int32,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [746]:
# Save the dataframe as a Pandas dataframe named pd_feature
pd_feature = dd_feat.compute()

NotImplementedError: Partition size is less than overlapping window size. Try using ``df.repartition`` to increase the partition size.

In [ ]:

pd_feature["Moving_Average_onReturns(10)"] = pd_feature.groupby('ticker')['returns'].rolling(10).mean()


In [ ]:
pd_feature.head()

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

It was not necessary to convert to pandas in order to calculate the moving average return. However, it would take much longer to compute the moving average in Dask and therefore is more expensive. 

Adding the "Moving_Average_onReturns(10)" column took  in Pandas. Calculating this in Dask took more than 5 minutes.

Additionally, the code in Pandas is much clearer, as adding a column is more direct. In Dask, to add this column we would need to use the assign() and groupby() functions, as well as a lambda function.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.